# Nuclear chromatin phenotypes of PBMCs reflects the treatment effect of proton therapy (all cancers)

---
This notebook summarizes the analysis corresponding to the results presented in figure 4 of the paper for all cancer patients. It can be used to rerun the analysis and regenerate the corresponding panels.

---

## 0. Environmental setup

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import matplotlib as mpl
from matplotlib.collections import PolyCollection
from matplotlib.legend_handler import HandlerTuple
from matplotlib.colors import to_rgb
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score

mpl.rcParams["figure.dpi"] = 1200

# SMALL_SIZE = 16
# MEDIUM_SIZE = 18
# BIGGER_SIZE = 20

# mpl.rc("font", size=SMALL_SIZE, weight="normal")  # controls default text sizes
# mpl.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
# mpl.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
# mpl.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
# mpl.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
# mpl.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
# mpl.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

import sys

sys.path.append("../..")
from src.utils.notebooks.eda import *
from src.utils.notebooks.figure3 import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

seed = 1234
random.seed(1234)
np.random.seed(1234)

%reload_ext nb_black

In [ ]:
nuc_feature_desc = pd.read_csv(
    "../../data/chrometric_feature_description.csv", index_col=0
)
feature_name_dict = dict(
    zip(
        list(nuc_feature_desc.loc[:, "feature"]),
        list(nuc_feature_desc.loc[:, "long_name"]),
    )
)
feature_color_dict = {
    "morphology": "b",
    "intensity": "g",
    "boundary": "r",
    "texture": "c",
    "chromatin condensation": "m",
    "moments": "y",
    np.nan: "k",
}
feature_color_dict = {
    feature: feature_color_dict[category]
    for (feature, category) in zip(
        list(nuc_feature_desc.loc[:, "long_name"]),
        list(nuc_feature_desc.loc[:, "category"]),
    )
}

In [ ]:
def plot_timepoint_cancer_markers_dist(
    data,
    markers,
    marker_labels,
    quantiles=None,
    cut=2,
    plot_type="violin",
    palette=None,
    figsize=[4, 4],
    hue=None,
    hue_order=None,
):
    mg_colors = ["lightsteelblue", "royalblue", "midnightblue"]
    gl_colors = ["orange", "gold", "saddlebrown"]
    hn_colors = ["plum", "deeppink", "indigo"]

    violin_all_colors = mg_colors + gl_colors + hn_colors
    bar_all_colors = []
    for i in range(3):
        bar_all_colors.append(mg_colors[i])
        bar_all_colors.append(gl_colors[i])
        bar_all_colors.append(hn_colors[i])

    for i in range(len(markers)):
        fig, ax = plot_marker_distribution(
            data,
            figsize=figsize,
            marker=markers[i],
            label_col="cancer",
            order=["Meningioma", "Glioma", "Head & Neck"],
            box_pairs=[
                (("Meningioma", "prior"), ("Meningioma", "during")),
                (("Meningioma", "prior"), ("Meningioma", "end")),
                (("Meningioma", "during"), ("Meningioma", "end")),
                (("Glioma", "prior"), ("Glioma", "during")),
                (("Glioma", "prior"), ("Glioma", "end")),
                (("Glioma", "during"), ("Glioma", "end")),
                (("Head & Neck", "prior"), ("Head & Neck", "during")),
                (("Head & Neck", "prior"), ("Head & Neck", "end")),
                (("Head & Neck", "during"), ("Head & Neck", "end")),
            ],
            quantiles=quantiles,
            cut=cut,
            plot_type=plot_type,
            palette="gray",
            hue="timepoint",
            hue_order=["prior", "during", "end"],
        )
        ax.set_xlabel("Cancer type")
        ax.set_ylabel(marker_labels[i])

        for label in ax.get_xticklabels():
            label.set_fontweight("bold")

        handles = []
        if plot_type == "violin":
            for ind, obj in enumerate(ax.findobj(PolyCollection)):
                rgb = to_rgb(violin_all_colors[ind])
                obj.set_facecolor(rgb)
                handles.append(
                    plt.Rectangle((0, 0), 0, 0, facecolor=rgb, edgecolor="black")
                )
        elif plot_type == "bar":
            for ind, bar in enumerate(ax.patches):
                rgb = to_rgb(bar_all_colors[ind])
                bar.set_color(rgb)
                handles_color = to_rgb(violin_all_colors[ind])
                handles.append(
                    plt.Rectangle(
                        (0, 0), 0, 0, facecolor=handles_color, edgecolor="black"
                    )
                )
        else:
            raise NotImplementedError

        ax.legend(
            handles=[tuple(handles[::3]), tuple(handles[1::3]), tuple(handles[2::3])],
            labels=tp_order,
            title=None,
            handlelength=4,
            handler_map={tuple: HandlerTuple(ndivide=None, pad=0)},
            loc="lower center",
            borderaxespad=0,
            bbox_to_anchor=(0.5, 1),
            ncol=3,
            frameon=False,
            prop={"size": 20, "weight": "bold"},
        )
        plt.show()
        plt.close()

In [ ]:
color_palette = {
    "Meningioma": "cornflowerblue",
    "Glioma": "orange",
    "Head & Neck": "orchid",
}

In [ ]:
mg_colors = ["lightsteelblue", "royalblue", "midnightblue"]
gl_colors = ["orange", "gold", "saddlebrown"]
hn_colors = ["plum", "deeppink", "indigo"]

---

## 1. Read in data

In this notebook we assess the differences of the cell states of PBMCs at three different time points of the proton therapy treatment: before, during (~3 weeks in) and at the end of the treatment (final week of treatment). To this end, we obtained PBMCs of 8 Meningioma, 8 Glioma and 6 Head & Neck cancer patients stained them for DNA, gH2AX and Lamin A/C and obtained fluorescent images.

First, we read in the required data set that describe each PBMCs by a number of hand-crafted features extracted from the fluorescent images of the cells.

#### Meningioma

In [ ]:
mg_tp1_root_data_dir = "../../data/meningioma/proteins/timepoint_1"
feature_file_path = "/preprocessed/full_pipeline/nuclear_features.csv"
qc_result_file_path = "/preprocessed/full_pipeline/segmentation/qc_results.csv"
gh2ax_foci_result_file_path = "/preprocessed/full_pipeline/gh2ax_foci_feats.csv"

In [ ]:
mg_tp1_data = read_in_protein_dataset(
    data_dir=mg_tp1_root_data_dir,
    feature_file_path=feature_file_path,
    qc_result_file_path=qc_result_file_path,
    gh2ax_foci_result_file_path=gh2ax_foci_result_file_path,
)
mg_tp1_data["id"] = mg_tp1_data["sample"] + mg_tp1_data["timepoint"]
mg_tp1_data["timepoint"] = "prior"
mg_tp1_data["cancer"] = "Meningioma"
mg_tp1_data["file_name"] = np.array(list(mg_tp1_data.index))

In [ ]:
mg_tp2_root_data_dir = "../../data/meningioma/proteins/timepoint_2"

mg_tp2_data = read_in_protein_dataset(
    data_dir=mg_tp2_root_data_dir,
    feature_file_path=feature_file_path,
    qc_result_file_path=qc_result_file_path,
    gh2ax_foci_result_file_path=gh2ax_foci_result_file_path,
)
mg_tp2_data["id"] = mg_tp2_data["sample"] + mg_tp2_data["timepoint"]
mg_tp2_data["timepoint"] = "during"
mg_tp2_data["cancer"] = "Meningioma"
mg_tp2_data["file_name"] = np.array(list(mg_tp2_data.index))

In [ ]:
mg_tp3_root_data_dir = "../../data/meningioma/proteins/timepoint_3"

mg_tp3_data = read_in_protein_dataset(
    data_dir=mg_tp3_root_data_dir,
    feature_file_path=feature_file_path,
    qc_result_file_path=qc_result_file_path,
    gh2ax_foci_result_file_path=gh2ax_foci_result_file_path,
)
mg_tp3_data["id"] = mg_tp3_data["sample"] + mg_tp3_data["timepoint"]
mg_tp3_data["timepoint"] = "end"
mg_tp3_data["cancer"] = "Meningioma"
mg_tp3_data["file_name"] = np.array(list(mg_tp3_data.index))

---

#### Glioma

In [ ]:
gl_tp1_root_data_dir = "../../data/glioma/proteins/timepoint_1"

gl_tp1_data = read_in_protein_dataset(
    data_dir=gl_tp1_root_data_dir,
    feature_file_path=feature_file_path,
    qc_result_file_path=qc_result_file_path,
    gh2ax_foci_result_file_path=gh2ax_foci_result_file_path,
)
gl_tp1_data["id"] = gl_tp1_data["sample"] + gl_tp1_data["timepoint"]
gl_tp1_data["timepoint"] = "prior"
gl_tp1_data["cancer"] = "Glioma"
gl_tp1_data["file_name"] = np.array(list(gl_tp1_data.index))

In [ ]:
gl_tp2_root_data_dir = "../../data/glioma/proteins/timepoint_2"

gl_tp2_data = read_in_protein_dataset(
    data_dir=gl_tp2_root_data_dir,
    feature_file_path=feature_file_path,
    qc_result_file_path=qc_result_file_path,
    gh2ax_foci_result_file_path=gh2ax_foci_result_file_path,
)
gl_tp2_data["id"] = gl_tp2_data["sample"] + gl_tp2_data["timepoint"]
gl_tp2_data["timepoint"] = "during"
gl_tp2_data["cancer"] = "Glioma"
gl_tp2_data["file_name"] = np.array(list(gl_tp2_data.index))

In [ ]:
gl_tp3_root_data_dir = "../../data/glioma/proteins/timepoint_3"

gl_tp3_data = read_in_protein_dataset(
    data_dir=gl_tp3_root_data_dir,
    feature_file_path=feature_file_path,
    qc_result_file_path=qc_result_file_path,
    gh2ax_foci_result_file_path=gh2ax_foci_result_file_path,
)
gl_tp3_data["id"] = gl_tp3_data["sample"] + gl_tp3_data["timepoint"]
gl_tp3_data["timepoint"] = "end"
gl_tp3_data["cancer"] = "Glioma"
gl_tp3_data["file_name"] = np.array(list(gl_tp3_data.index))

---

#### Head & Neck cancers

In [ ]:
hn_tp1_root_data_dir = "../../data/headneck/proteins/timepoint_1"

hn_tp1_data = read_in_protein_dataset(
    data_dir=hn_tp1_root_data_dir,
    feature_file_path=feature_file_path,
    qc_result_file_path=qc_result_file_path,
    gh2ax_foci_result_file_path=gh2ax_foci_result_file_path,
)
hn_tp1_data["id"] = hn_tp1_data["sample"] + hn_tp1_data["timepoint"]
hn_tp1_data["timepoint"] = "prior"
hn_tp1_data["cancer"] = "Head & Neck"
hn_tp1_data["file_name"] = np.array(list(hn_tp1_data.index))

In [ ]:
hn_tp2_root_data_dir = "../../data/headneck/proteins/timepoint_2"

hn_tp2_data = read_in_protein_dataset(
    data_dir=hn_tp2_root_data_dir,
    feature_file_path=feature_file_path,
    qc_result_file_path=qc_result_file_path,
    gh2ax_foci_result_file_path=gh2ax_foci_result_file_path,
)
hn_tp2_data["id"] = hn_tp2_data["sample"] + hn_tp2_data["timepoint"]
hn_tp2_data["timepoint"] = "during"
hn_tp2_data["cancer"] = "Head & Neck"
hn_tp2_data["file_name"] = np.array(list(hn_tp2_data.index))

In [ ]:
hn_tp3_root_data_dir = "../../data/headneck/proteins/timepoint_3"

hn_tp3_data = read_in_protein_dataset(
    data_dir=hn_tp3_root_data_dir,
    feature_file_path=feature_file_path,
    qc_result_file_path=qc_result_file_path,
    gh2ax_foci_result_file_path=gh2ax_foci_result_file_path,
)
hn_tp3_data["id"] = hn_tp3_data["sample"] + hn_tp3_data["timepoint"]
hn_tp3_data["timepoint"] = "end"
hn_tp3_data["cancer"] = "Head & Neck"
hn_tp3_data["file_name"] = np.array(list(hn_tp3_data.index))

---

## 2. Data preprocessing

Before we analyze the data, we will first preprocess it.

To this end, we first remove features with missing values and samples with missing features.

In [ ]:
mg_tp1_data = preprocess_data(mg_tp1_data, remove_constant_features=False)
mg_tp2_data = preprocess_data(mg_tp2_data, remove_constant_features=False)
mg_tp3_data = preprocess_data(mg_tp3_data, remove_constant_features=False)

gl_tp1_data = preprocess_data(gl_tp1_data, remove_constant_features=False)
gl_tp2_data = preprocess_data(gl_tp2_data, remove_constant_features=False)
gl_tp3_data = preprocess_data(gl_tp3_data, remove_constant_features=False)

hn_tp1_data = preprocess_data(hn_tp1_data, remove_constant_features=False)
hn_tp2_data = preprocess_data(hn_tp2_data, remove_constant_features=False)
hn_tp3_data = preprocess_data(hn_tp3_data, remove_constant_features=False)

all_data = (
    mg_tp1_data.append(gl_tp1_data)
    .append(hn_tp1_data)
    .append(mg_tp2_data)
    .append(gl_tp2_data)
    .append(hn_tp2_data)
    .append(gl_tp3_data)
    .append(hn_tp3_data)
    .append(mg_tp3_data)
)

all_data.index = all_data["id"] + "_" + all_data.index

In [ ]:
all_data = all_data.rename(columns=feature_name_dict)

We remove any duplicates.

In [ ]:
all_data = all_data.loc[~all_data.duplicated()]

In [ ]:
fig, ax = plt.subplots(figsize=[12, 4], ncols=2)
tp_order = ["prior", "during", "end"]
sample_order = np.unique(all_data.loc[:, "sample"])
ax = ax.flatten()
ax[0] = sns.countplot(
    x="sample",
    data=all_data,
    ax=ax[0],
    order=sample_order,
    hue_order=tp_order,
    hue="timepoint",
    palette="gray",
)
ax[0].legend([], [], frameon=False)
ax[0].set_xlabel("ID of the biological sample")
ax[0].set_title("Distribution of biological samples")
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)

ax[1] = sns.countplot(
    x="timepoint",
    hue="timepoint",
    data=all_data,
    ax=ax[1],
    order=tp_order,
    dodge=False,
    palette="gray",
)
ax[1].set_xlabel("Timepoint with respect to the treatment")
ax[1].set_title("Distribution of the different timepoints")
ax[1].legend(loc="lower right")

plt.show()
plt.close()

___

#### Subsampling

We next subsample the data set such that for each timepoint we have the same number of nuclei in the data set. Additionally, we ensure that the individual timepoint population are approximately uniformly represented by the different biological (patient) samples.

In [ ]:
sampled_data = get_stratified_data(
    all_data, id_column="id", cond_column="timepoint", seed=1234
)

In [ ]:
fig, ax = plt.subplots(figsize=[18, 4], ncols=3)
tp_order = ["prior", "during", "end"]
cancer_order = ["Meningioma", "Glioma", "Head & Neck"]
sample_order = np.unique(all_data.loc[:, "sample"])
ax = ax.flatten()
ax[0] = sns.countplot(
    x="sample",
    data=sampled_data,
    ax=ax[0],
    order=sample_order,
    hue_order=tp_order,
    hue="timepoint",
    palette="gray",
)
ax[0].legend([], [], frameon=False)
ax[0].set_xlabel("ID of the biological sample")
ax[0].set_title("Distribution of biological samples")
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)

ax[1] = sns.countplot(
    x="timepoint",
    hue="timepoint",
    data=sampled_data,
    ax=ax[1],
    order=tp_order,
    hue_order=tp_order,
    dodge=False,
    palette="gray",
)
ax[1].set_xlabel("Timepoint with respect to the treatment")
ax[1].set_title("Distribution of the different timepoints")
ax[1].legend(loc="lower right")

ax[2] = sns.countplot(
    x="cancer",
    hue="cancer",
    data=sampled_data,
    ax=ax[2],
    order=cancer_order,
    hue_order=cancer_order,
    dodge=False,
    palette=color_palette,
)
ax[2].set_xlabel("Cancer type")
ax[2].set_title("Distribution of the different cancer types")
ax[2].legend(loc="lower right")

plt.show()
plt.close()

----

#### Sample and feature selection

We now filter out constant features and nuclei with missing features.

In [ ]:
data = preprocess_data(sampled_data, remove_constant_features=True)

---

#### Data preparation

After sampling the data, we will now prepare the data for the consecutive analysis, i.e. extracting only chrometric features and corresponding metadata information.

In [ ]:
all_chrometric_data = get_chrometric_data(
    data,
    proteins=["gh2ax", "lamin", "cd3"],
    exclude_dna_int=True,
)

sample_labels = data.loc[:, "sample"]
tp_labels = data.loc[:, "timepoint"]
cancer_labels = data.loc[:, "cancer"]

Finally, we remove highly correlated features (Pearson $\rho > 0.8$) from the chrometric features.

In [ ]:
chrometric_data = remove_correlated_features(all_chrometric_data, threshold=0.8)

---

## 3. Panels

Now we generate the individual panels for figure 4 of the paper.

---

### 3b. Parametric analysis captures captures differences of PBMCs at different timepoints of the proton therapy treatment

The montage already indicate significant changes in particular between the chrometric pheontype of the PBMCs prior the treatment and the end of it. We will now turn to the assessment of the parametric descriptions of the nuclear phenotypes of the PBMCs at those different timepoints. To this end, we first visualize the data set using a tSNE plot to assess potential large-scale differences between the timepoint populations and individual patient samples.

In [ ]:
chrometric_embs = get_tsne_embs(chrometric_data)
chrometric_embs["timepoint"] = np.array(tp_labels)
chrometric_embs["sample"] = np.array(sample_labels)
chrometric_embs["cancer"] = np.array(cancer_labels)

In [ ]:
fig, ax = plt.subplots(figsize=[18, 16])
ax = sns.scatterplot(
    data=chrometric_embs,
    x="tSNE 1",
    y="tSNE 2",
    hue="cancer",
    hue_order=cancer_order,
    ax=ax,
    s=15,
    style="timepoint",
    style_order=tp_order,
    palette=color_palette,
)
ax.set_xlim([-50, 50])
ax.set_ylim([-55, 55])
plt.show()

Note that the above plot excluds one outlier for better visualization.

In [ ]:
fig, ax = plt.subplots(figsize=[18, 16])
ax = sns.scatterplot(
    data=chrometric_embs,
    x="tSNE 1",
    y="tSNE 2",
    hue="sample",
    style="timepoint",
    style_order=tp_order,
    hue_order=sample_order,
    ax=ax,
    s=15,
    marker="o",
    palette="tab20",
)
ax.set_xlim([-50, 50])
ax.set_ylim([-55, 55])
plt.legend(
    bbox_to_anchor=(1.02, 0.5), loc="center left", borderaxespad=0, title="sample"
)
plt.show()

The tSNE plot also shows that especially the chrometric phenotypes of the PBMCs at the intermediate timepoint of the proton therapy look different from the prior and the end of treatment population.
The patient samples are fairly well mixed with probably the small exception of the PBMCs of P29 and P22.

In [ ]:
cancer_groups = ["Meningioma", "Glioma", "Head & Neck"]
tps = ["prior", "during", "end"]
for cg in cancer_groups:
    for tp in tps:
        fig, ax = plt.subplots(figsize=[8, 6])
        ax = sns.scatterplot(
            data=chrometric_embs.loc[
                (chrometric_embs.cancer == cg) & (chrometric_embs.timepoint == tp)
            ],
            x="tSNE 1",
            y="tSNE 2",
            hue="sample",
            hue_order=np.unique(
                chrometric_embs.loc[
                    (chrometric_embs.cancer == cg) & (chrometric_embs.timepoint == tp),
                    "sample",
                ]
            ),
            ax=ax,
            s=12,
            marker="o",
            palette="tab10",
        )
        plt.legend(
            bbox_to_anchor=(0.5, 1.05),
            loc="center",
            borderaxespad=0,
            title="",
            ncol=10,
            fancybox=False,
            frameon=False,
            columnspacing=0.1,
        )
        ax.set_xlim([-60, 60])
        ax.set_ylim([-60, 60])
        ax.set_xlabel("")
        ax.set_ylabel("")
        plt.show()

---

#### Classification of the different cancer types.

To quantify the separability of the timepoint populations using the chrometric phenotypes of the PBMCs of the different cancer patients we perform a 10-fold stratified cross-validation analysis using a RandomForest classifier. The classifier provides a simple non-linear classification model which also yields an importance measure for the individual chrometric features indicating which ones are most different between the three populations.

##### Nuclei split

At first we will split the data randomly on a nuclei-basis, i.e. nuclei of the same patient will be likely included in both the training and the test sets.

In [ ]:
rfc = RandomForestClassifier(
    n_estimators=500, n_jobs=10, random_state=seed, class_weight="balanced"
)

In [ ]:
tp_cv_conf_mtx_nuclei = get_cv_conf_mtx(
    estimator=rfc,
    features=chrometric_data,
    labels=tp_labels,
    scale_features=False,
    n_folds=10,
    order=tp_order,
)
normalized_cv_conf_mtx_nuclei = tp_cv_conf_mtx_nuclei.divide(
    tp_cv_conf_mtx_nuclei.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_nuclei,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
    annot_kws={"size": 12, "weight": "bold"},
)
ax.set_xlabel("Predicted treatment timepoint")
ax.set_ylabel("True treatment timepoint")
plt.show()

In [ ]:
cancer_cv_conf_mtx_nuclei = get_cv_conf_mtx(
    estimator=rfc,
    features=chrometric_data,
    labels=cancer_labels,
    scale_features=False,
    n_folds=10,
    order=cancer_order,
)
normalized_cv_conf_mtx_nuclei = cancer_cv_conf_mtx_nuclei.divide(
    cancer_cv_conf_mtx_nuclei.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_nuclei,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
    annot_kws={"size": 12, "weight": "bold"},
)
ax.set_xlabel("Predicted cancer type")
ax.set_ylabel("True cancer type")
plt.show()

To further assess the similarity of the different timepoint  and cancer type distribution and their linear separability we run two linear discriminant analysis using a single component.

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=1)
lda_tp_cv_conf_mtx_nuclei = get_cv_conf_mtx(
    estimator=lda,
    features=chrometric_data,
    labels=tp_labels,
    scale_features=True,
    n_folds=10,
    order=tp_order,
)
lda_normalized_cv_conf_mtx_nuclei = lda_tp_cv_conf_mtx_nuclei.divide(
    lda_tp_cv_conf_mtx_nuclei.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    lda_normalized_cv_conf_mtx_nuclei,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
    annot_kws={"size": 12, "weight": "bold"},
)
ax.set_xlabel("Predicted treatment timepoint")
ax.set_ylabel("True treatment timepoint")
plt.show()

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=1)
lda_cancer_cv_conf_mtx_nuclei = get_cv_conf_mtx(
    estimator=lda,
    features=chrometric_data,
    labels=cancer_labels,
    scale_features=True,
    n_folds=10,
    order=cancer_order,
)
lda_normalized_cv_conf_mtx_nuclei = lda_cancer_cv_conf_mtx_nuclei.divide(
    lda_cancer_cv_conf_mtx_nuclei.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    lda_normalized_cv_conf_mtx_nuclei,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
    annot_kws={"size": 12, "weight": "bold"},
)
ax.set_xlabel("Predicted cancer type")
ax.set_ylabel("True cancer type")
plt.show()

In [ ]:
lda_transformed = pd.DataFrame(
    lda.fit(chrometric_data, tp_labels).transform(chrometric_data),
    columns=["LD 1 (timepoint)"],
    index=chrometric_data.index,
)
lda_transformed["LD 1 (cancer type)"] = lda.fit(
    chrometric_data, cancer_labels
).transform(chrometric_data)
lda_transformed["timepoint"] = np.array(tp_labels)
lda_transformed["sample"] = np.array(sample_labels)
lda_transformed["cancer"] = np.array(cancer_labels)
g = sns.jointplot(
    data=lda_transformed,
    x="LD 1 (timepoint)",
    y="LD 1 (cancer type)",
    hue="cancer",
    s=2,
    hue_order=cancer_order,
    height=6,
    palette=color_palette,
    xlim=[-4, 4],
    ylim=[-4, 4],
)

In [ ]:
g = sns.jointplot(
    data=lda_transformed,
    x="LD 1 (timepoint)",
    y="LD 1 (cancer type)",
    hue="timepoint",
    s=2,
    hue_order=tp_order,
    height=6,
    palette="gray",
    xlim=[-4, 4],
    ylim=[-4, 4],
)

In [ ]:
np.sum(np.abs(lda_transformed.loc[:, "LD 1 (timepoint)"]) > 4) + np.sum(
    np.abs(lda_transformed.loc[:, "LD 1 (cancer type)"]) > 4
)

In [ ]:
len(lda_transformed)

Note that in the above plot 102/21600 outlier sample is excluded for better visualization of the population differences.

---
#### Patient split


While the previous analysis assess the level of differences of the chrometric phenotypes of the PBMCs between the different treatment timepoints, the classifier can make use of patient specific characteristics during the classification. In a diagnostic use case such information would not be available. To evaluate how well a classifier would be able to predict for unseen patient the corresponding treatment timepoint simply based on the chrometric phenotypes of the PBMCs, we also assess the class separability using a stratified 10-fold patient-cross-validation approach. Thereby at each iteration 1 of the patients of each timepoint are hold out for the test set.

In [ ]:
tp_cv_conf_mtx_patient = get_cv_conf_mtx(
    estimator=rfc,
    features=chrometric_data,
    labels=tp_labels,
    groups=sample_labels,
    scale_features=False,
    n_folds=10,
    order=tp_order,
)
normalized_cv_conf_mtx_patient = tp_cv_conf_mtx_patient.divide(
    tp_cv_conf_mtx_patient.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_patient,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
)
ax.set_xlabel("Predicted treatment timepoint")
ax.set_ylabel("True treatment timepoint")
plt.show()

In [ ]:
cancer_cv_conf_mtx_patient = get_cv_conf_mtx(
    estimator=rfc,
    features=chrometric_data,
    labels=cancer_labels,
    groups=sample_labels,
    scale_features=False,
    n_folds=10,
    order=cancer_order,
)
normalized_cv_conf_mtx_patient = cancer_cv_conf_mtx_patient.divide(
    cancer_cv_conf_mtx_patient.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_patient,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
)
ax.set_xlabel("Predicted cancer type")
ax.set_ylabel("True cancer type")
plt.show()

In general the confusion matrix again shows a similar picture as seen for the nuclei-based split showing that end of treatment population show phenotypes of the PBMCs significantly different from the other two treatment timepoints.

---

### Cancer type separability over the treatment course

We notice that the separability between the different cancer types is significantly smaller than what we observed when only considering samples prior the treatment. We thus now assess similarly the separability of the different cancer types for each of the treatment timepoints individually.

#### Prior treatment: Leave-on-per-category-out

In [ ]:
prior_idc = tp_labels.loc[tp_labels == "prior"].index
prior_cancer_labels = cancer_labels.loc[prior_idc]
prior_chrometric_data = chrometric_data.loc[prior_idc]
prior_sample_labels = sample_labels.loc[prior_idc]

In [ ]:
prior_cancer_cv_conf_mtx_patient = get_cv_conf_mtx(
    estimator=rfc,
    features=prior_chrometric_data,
    labels=prior_cancer_labels,
    groups=prior_sample_labels,
    scale_features=False,
    n_folds=10,
    order=cancer_order,
)
normalized_cv_conf_mtx_patient = prior_cancer_cv_conf_mtx_patient.divide(
    prior_cancer_cv_conf_mtx_patient.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_patient,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
)
ax.set_xlabel("Predicted cancer type")
ax.set_ylabel("True cancer type")
plt.show()

In [ ]:
groupkfold = StratifiedGroupKFold(n_splits=10)
prior_cv_bacs = cross_val_score(
    rfc,
    cv=groupkfold,
    X=prior_chrometric_data,
    y=prior_cancer_labels,
    groups=prior_sample_labels,
    scoring="balanced_accuracy",
    n_jobs=10,
)
print(
    "Balanced accuracy: {} (+/- {})".format(
        np.mean(prior_cv_bacs), np.std(prior_cv_bacs)
    )
)

___

#### During: Leave-one-per-category-out

In [ ]:
during_idc = tp_labels.loc[tp_labels == "during"].index
during_cancer_labels = cancer_labels.loc[during_idc]
during_chrometric_data = chrometric_data.loc[during_idc]
during_sample_labels = sample_labels.loc[during_idc]

In [ ]:
during_cancer_cv_conf_mtx_patient = get_cv_conf_mtx(
    estimator=rfc,
    features=during_chrometric_data,
    labels=during_cancer_labels,
    groups=during_sample_labels,
    scale_features=False,
    n_folds=10,
    order=cancer_order,
)
normalized_cv_conf_mtx_patient = during_cancer_cv_conf_mtx_patient.divide(
    during_cancer_cv_conf_mtx_patient.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_patient,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
)
ax.set_xlabel("Predicted cancer type")
ax.set_ylabel("True cancer type")
plt.show()

In [ ]:
groupkfold = StratifiedGroupKFold(n_splits=10)
during_cv_bacs = cross_val_score(
    rfc,
    cv=groupkfold,
    X=during_chrometric_data,
    y=during_cancer_labels,
    groups=during_sample_labels,
    scoring="balanced_accuracy",
    n_jobs=10,
)
print(
    "Balanced accuracy: {} (+/- {})".format(
        np.mean(during_cv_bacs), np.std(during_cv_bacs)
    )
)

---

#### End of therapy: Leave-one-per-category out

In [ ]:
end_idc = tp_labels.loc[tp_labels == "end"].index
end_cancer_labels = cancer_labels.loc[end_idc]
end_chrometric_data = chrometric_data.loc[end_idc]
end_sample_labels = sample_labels.loc[end_idc]

In [ ]:
end_cancer_cv_conf_mtx_patient = get_cv_conf_mtx(
    estimator=rfc,
    features=end_chrometric_data,
    labels=end_cancer_labels,
    groups=end_sample_labels,
    scale_features=False,
    n_folds=10,
    order=cancer_order,
)
normalized_cv_conf_mtx_patient = end_cancer_cv_conf_mtx_patient.divide(
    end_cancer_cv_conf_mtx_patient.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_patient,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
)
ax.set_xlabel("Predicted cancer type")
ax.set_ylabel("True cancer type")
plt.show()

In [ ]:
groupkfold = StratifiedGroupKFold(n_splits=10)
end_cv_bacs = cross_val_score(
    rfc,
    cv=groupkfold,
    X=end_chrometric_data,
    y=end_cancer_labels,
    groups=end_sample_labels,
    scoring="balanced_accuracy",
    n_jobs=10,
)
print(
    "Balanced accuracy: {} (+/- {})".format(np.mean(end_cv_bacs), np.std(end_cv_bacs))
)

---
#### Summary: Leave-one-per-category out

In [ ]:
cv_bacs_summary = pd.DataFrame(
    np.concatenate([prior_cv_bacs, during_cv_bacs, end_cv_bacs]), columns=["BAC"]
)
cv_bacs_summary["timepoint"] = (
    ["prior"] * len(prior_cv_bacs)
    + ["during"] * len(during_cv_bacs)
    + ["end"] * len(end_cv_bacs)
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.violinplot(data=cv_bacs_summary, x="timepoint", y="BAC", palette="gray")
annotator = Annotator(
    ax,
    [("prior", "during"), ("prior", "end"), ("during", "end")],
    data=cv_bacs_summary,
    x="timepoint",
    y="BAC",
    order=["prior", "during", "end"],
    plot="violinplot",
)
annotator.configure(
    test="Wilcoxon",
    text_format="star",
    loc="inside",
    comparisons_correction="Benjamini-Hochberg",
)
annotator.apply_test()
annotator.annotate()
plt.show()
plt.close()

---

As an alternative to the previous procedure, we also evaluate the separability of the cancer types at each treatment timepoint using a leave-one-patient-out cross-validation scheme. Thereby, the training fold data is randomly subsampled to ensure equal representation of the different cancer types. To obtain a random baseline, we also apply the procedure to a setup, where the individual cancer type labels were randomly permuted for 10 times.

#### Prior: Leave-one-patient-out

In [ ]:
lopo_prior_cv_result = summarize_group_cv_results_by_fold(
    model=rfc,
    features=prior_chrometric_data,
    labels=prior_cancer_labels,
    groups=prior_sample_labels,
    balance_train=True,
)

np.random.seed(seed)
bs = range(10)

lopo_prior_perm_cv_results = []

for b in tqdm(bs):
    prior_perm_cancer_labels = get_permute_group_labels(
        prior_cancer_labels, prior_sample_labels
    )[0]
    lopo_perm_cv_result = summarize_group_cv_results_by_fold(
        model=rfc,
        features=prior_chrometric_data,
        labels=prior_perm_cancer_labels,
        groups=prior_sample_labels,
        balance_train=True,
    )
    lopo_perm_cv_result["permutation"] = b
    lopo_prior_perm_cv_results.append(lopo_perm_cv_result)

lopo_prior_perm_cv_results = pd.concat(lopo_prior_perm_cv_results)
lopo_prior_perm_cv_results["condition"] = "Permuted"
lopo_prior_cv_result["condition"] = "Observed"
all_lopo_prior_results = lopo_prior_cv_result.append(lopo_prior_perm_cv_results)
all_lopo_prior_results["timepoint"] = "prior"

In [ ]:
fig, ax = plot_lopo_cv_results_by_class(
    all_lopo_prior_results,
    cancer_order,
    x="majority_class",
    y="score",
    hue="condition",
    figsize=[6, 4],
    test="Mann-Whitney",
    pval_text_format="star",
    alpha=0.5,
)
ax.set_xlabel("Cancer types")
ax.set_ylabel("Classification accuracy by patient")
plt.show()

---

#### During: Leave-one-patient-out

In [ ]:
lopo_during_cv_result = summarize_group_cv_results_by_fold(
    model=rfc,
    features=during_chrometric_data,
    labels=during_cancer_labels,
    groups=during_sample_labels,
    balance_train=True,
)

np.random.seed(seed)
bs = range(10)

lopo_during_perm_cv_results = []

for b in tqdm(bs):
    during_perm_cancer_labels = get_permute_group_labels(
        during_cancer_labels, during_sample_labels
    )[0]
    lopo_perm_cv_result = summarize_group_cv_results_by_fold(
        model=rfc,
        features=during_chrometric_data,
        labels=during_perm_cancer_labels,
        groups=during_sample_labels,
        balance_train=True,
    )
    lopo_perm_cv_result["permutation"] = b
    lopo_during_perm_cv_results.append(lopo_perm_cv_result)

lopo_during_perm_cv_results = pd.concat(lopo_during_perm_cv_results)
lopo_during_perm_cv_results["condition"] = "Permuted"
lopo_during_cv_result["condition"] = "Observed"
all_lopo_during_results = lopo_during_cv_result.append(lopo_during_perm_cv_results)
all_lopo_during_results["timepoint"] = "during"

In [ ]:
fig, ax = plot_lopo_cv_results_by_class(
    all_lopo_during_results,
    cancer_order,
    x="majority_class",
    y="score",
    hue="condition",
    figsize=[6, 4],
    test="Mann-Whitney",
    pval_text_format="star",
    alpha=0.5,
)
ax.set_xlabel("Cancer types")
ax.set_ylabel("Classification accuracy by patient")
plt.show()

---
#### End of treatment: Leave-one-patient-out

In [ ]:
lopo_end_cv_result = summarize_group_cv_results_by_fold(
    model=rfc,
    features=end_chrometric_data,
    labels=end_cancer_labels,
    groups=end_sample_labels,
    balance_train=True,
)

np.random.seed(seed)
bs = range(10)

lopo_end_perm_cv_results = []

for b in tqdm(bs):
    end_perm_cancer_labels = get_permute_group_labels(
        end_cancer_labels, end_sample_labels
    )[0]
    lopo_perm_cv_result = summarize_group_cv_results_by_fold(
        model=rfc,
        features=end_chrometric_data,
        labels=end_perm_cancer_labels,
        groups=end_sample_labels,
        balance_train=True,
    )
    lopo_perm_cv_result["permutation"] = b
    lopo_end_perm_cv_results.append(lopo_perm_cv_result)

lopo_end_perm_cv_results = pd.concat(lopo_end_perm_cv_results)
lopo_end_perm_cv_results["condition"] = "Permuted"
lopo_end_cv_result["condition"] = "Observed"
all_lopo_end_results = lopo_end_cv_result.append(lopo_end_perm_cv_results)
all_lopo_end_results["timepoint"] = "end"

In [ ]:
fig, ax = plot_lopo_cv_results_by_class(
    all_lopo_end_results,
    cancer_order,
    x="majority_class",
    y="score",
    hue="condition",
    figsize=[6, 4],
    test="Mann-Whitney",
    pval_text_format="star",
    alpha=0.5,
)
ax.set_xlabel("Cancer types")
ax.set_ylabel("Classification accuracy by patient")
plt.show()

#### Summary: Leave-one-patient-out

In [ ]:
all_lopo_tp_results = all_lopo_prior_results.append(all_lopo_during_results).append(
    all_lopo_end_results
)
all_lopo_tp_results.loc[:, "tp"] = all_lopo_tp_results.loc[:, "timepoint"]
all_lopo_tp_results.loc[
    all_lopo_tp_results.condition == "Permuted", "tp"
] = "permutation"

In [ ]:
fig, ax = plot_lopo_cv_results_by_class(
    all_lopo_tp_results,
    tp_order,
    x="timepoint",
    y="score",
    hue="condition",
    figsize=[6, 4],
    test="Mann-Whitney",
    pval_text_format="star",
    alpha=0.5,
)
ax.set_xlabel("Treatment timepoint")
ax.set_ylabel("Classification accuracy by patient")
plt.show()

In [ ]:
fig, ax = plot_lopo_cv_results_timepoints(
    all_lopo_tp_results,
    order=["prior", "during", "end", "permutation"],
    class_palette=color_palette,
    box_palette=["lightgray"],
    figsize=[6, 6],
)
ax.set_ylabel("Classification accuracy")
ax.set_xlabel("Timepoint")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=[3.5, 8])
ax = sns.barplot(
    data=all_lopo_tp_results,
    x="tp",
    y="score",
    order=["prior", "during", "end", "permutation"],
    # fliersize=2,
    palette=["gray"],
    ax=ax,
)

box_pairs = [
    ("prior", "during"),
    ("during", "end"),
    ("prior", "end"),
    ("prior", "permutation"),
    ("during", "permutation"),
    ("end", "permutation"),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=all_lopo_tp_results,
    x="tp",
    y="score",
    order=["prior", "during", "end", "permutation"],
    plot="barplot",
)
annotator.configure(
    test="Mann-Whitney",
    text_format="star",
    loc="inside",
    comparisons_correction="Benjamini-Hochberg",
)
annotator.apply_test()
annotator.annotate()
ax.set_xlabel("")
ax.set_ylabel("")
# ax.set_ylim([0, 1])
# fig.savefig(
#     os.path.join(output_dir, "cancer_separation_accuracy.png"),
#     dpi=1200,
#     transparent=True,
# )
plt.show()

---

### 3c. Nuclear chromatin biomarkers identifying cancer populations

#### Feature importance

We have validated that there are significant differences between the individual treatment timepoints in particular when comparing PBMCs at the end of the treatment with those of during respectively prior the treatment. We next assess the implicit feature importance of a RandomForest classifier trained on the task to distinguish between the timepoint populations types in order to get an idea of the features are most indicative for the treatment effect.

In [ ]:
fig, ax = plot_feature_importance_for_estimator(
    rfc,
    chrometric_data,
    tp_labels,
    cmap=["gray"],
    figsize=[2, 3],
    feature_color_dict=feature_color_dict,
    n_features=15,
)


In [ ]:
fig, ax = plot_feature_importance_for_estimator(
    rfc,
    chrometric_data,
    cancer_labels,
    cmap=["gray"],
    figsize=[2, 3],
    feature_color_dict=feature_color_dict,
    n_features=15,
)

In [ ]:
marker_screen_results_tp = find_markers(chrometric_data, tp_labels)
marker_screen_results_cancer = find_markers(chrometric_data, cancer_labels)

#### Prior treatment

In [ ]:
marker_screen_results_tp.loc[marker_screen_results_tp.label == "prior"].head(10)

---

#### During treatment

Next we look at the features whose mean is significantly different in the during treatment population.

In [ ]:
marker_screen_results_tp.loc[marker_screen_results_tp.label == "during"].head(10)

---

#### End of treatment

Finally, we also evaluate the chrometric phenotype of PBMCs at the end of the proton therapy treatment.

In [ ]:
marker_screen_results_tp.loc[marker_screen_results_tp.label == "end"].head(10)

---

#### Meningioma

In [ ]:
marker_screen_results_cancer.loc[
    marker_screen_results_cancer.label == "Meningioma"
].head(10)

#### Glioma

In [ ]:
marker_screen_results_cancer.loc[marker_screen_results_cancer.label == "Glioma"].head(
    10
)

#### Head & Neck cancers

In [ ]:
marker_screen_results_cancer.loc[
    marker_screen_results_cancer.label == "Head & Neck"
].head(10)

In [ ]:
color_palette

---

In [ ]:
markers = [
    "volume",
    "hetero_to_euchromatin_area_ratio",
    "radial_chromatin_content_p10",
    "std_curvature",
    "glcm_contrast_5px",
    "glcm_correlation_5px",
]
marker_labels = [
    r"Nuclear volume in px$^3$",
    "relative HC/EC ratio",
    "Fraction of the overall DNA intensity \n within the inner 10% of the nuclear volume",
    "Standard deviation of the curvature",
    "Contrast of the GLCM \n with a shift of 5px (2D)",
    "Correlation of the GLCM \n with a shift of 5px (2D)",
]
plot_timepoint_cancer_markers_dist(
    data, markers, marker_labels, cut=0, palette=color_palette, figsize=[6.5, 3.5]
)

In [ ]:
plot_timepoint_cancer_markers_dist(
    data,
    markers,
    marker_labels,
    cut=0,
    plot_type="bar",
    palette=color_palette,
    figsize=[5, 4],
)

---

### 3d. Proteomic differences of PBMCs in cancer

Finally, we also assess the proteomic differences between the different treatment timepoint populations. To this end, we plot the relative Lamin and gH2AX expression measured by the sum of the intensities of the corresponding imaging channels normalized by the nuclear volume. Additionally, we plot the number of identified gH2AX foci which are computed as the local maxima peaks found in the corresponding channel images.

Note that those features are only available for the first data set that was stained for those proteins.

In [ ]:
markers = [
    "rel_lamin_3d_int",
    "rel_gh2ax_3d_int",
    "gh2ax_foci_count",
    "gh2ax_sum_foci_area",
    "gh2ax_avg_foci_area",
]
marker_labels = [
    "Volume-normalized nuclear\nLamin A/C intensity",
    "Normalized nuclear\n" r"$\gamma$H2AX intensity",
    r"Number of $\gamma$H2AX foci",
    r"Sum of the $\gamma$H2AX foci area",
    r"Average size of the $\gamma$H2AX foci",
]
plot_timepoint_cancer_markers_dist(
    data,
    markers,
    marker_labels,
    quantiles=None,
    cut=0,
    plot_type="bar",
    palette=color_palette,
    figsize=[6, 4],
)

In [ ]:
markers = [
    "rel_lamin_3d_int",
    "rel_gh2ax_3d_int",
    "gh2ax_foci_count",
    "gh2ax_sum_foci_area",
    "gh2ax_avg_foci_area",
]
marker_labels = [
    "Volume-normalized nuclear\nLamin A/C intensity",
    "Normalized nuclear\n" r"$\gamma$H2AX intensity",
    r"Number of $\gamma$H2AX foci",
    r"Sum of the $\gamma$H2AX foci area",
    r"Average size of the $\gamma$H2AX foci",
]
plot_timepoint_cancer_markers_dist(
    data,
    markers,
    marker_labels,
    quantiles=None,
    cut=0,
    plot_type="violin",
    palette=color_palette,
    figsize=[5, 4],
)

---

## 4. Supplemental

### 4.1. Toxicity assessment

Hereinafter we assess the toxicity of the treatment for the selected cancer patient population. To this end, we use an assessment of the well-being of the patients during/after the treatment using the CTCAE terminology and grading scheme.

In [ ]:
tox_data = pd.read_csv("../../data/patient_toxicity.csv")
tox_data.patient_id = "p" + tox_data.loc[:, "patient_id"].astype("str")
tox_data = tox_data.loc[tox_data.patient_id.isin(np.unique(data.loc[:, "sample"]))]
tox_data["cancer"] = "Meningioma"
tox_data.loc[
    tox_data.patient_id.isin(data.loc[data.cancer == "Glioma", "sample"]), "cancer"
] = "Glioma"
tox_data.loc[
    tox_data.patient_id.isin(data.loc[data.cancer == "Head & Neck", "sample"]), "cancer"
] = "Head & Neck"
tox_data["fatigue"] = "no"
tox_data.head()

In [ ]:
set(sample_labels) - set(np.unique(tox_data.patient_id))

---
#### 4.1.a. EDA

A closer look at the data shows that for all except for the Meningioma patient P52 CTCAE assessment information are available.
We now will look at the distribution of a number of key characteristics which are of special interest from a clinical perspective. Namely, the occurence of fatigue, skin lesion (in particular dermatitis) and in general the grade distribution.

In [ ]:
fig, ax = plt.subplots(figsize=[6, 4])
ax = sns.countplot(data=tox_data, x="grade", hue="cancer", palette=color_palette)
ax.set_yticks(np.arange(0, 30, 2))
plt.legend(loc="upper right")
plt.show()
plt.close()

To assess if there are significant differences between the PBMC population of cancer patients that experienced fatigue during the treatment and/or dermatitis, or any other severe symptom (grade >= 3), we add corresponding labels to the PBMC data. Note that all labels are binary and thus neglect the different grades for dermatitis.

In [ ]:
data.loc[:, "fatigue"] = 0
data.loc[:, "dermatitis radiation"] = 0
data.loc[:, "toxic"] = "no"
for cat in ["fatigue", "dermatitis radiation"]:
    cat_patients = np.unique(tox_data.loc[tox_data.category == cat, "patient_id"])
    for cat_patient in cat_patients:
        data.loc[data.loc[:, "sample"] == cat_patient, cat] = np.array(
            tox_data.loc[
                (tox_data.patient_id == cat_patient) & (tox_data.category == cat),
                "grade",
            ]
        )[0]

toxic_patients = np.unique(tox_data.loc[tox_data.grade > 2, "patient_id"])
for toxic_patient in toxic_patients:
    data.loc[data.loc[:, "sample"] == toxic_patient, "toxic"] = "yes"

---

#### 4.1.b. Assessment of the chrometric differences for patients with high toxicity

We first, will assess the distribution of the previously identified chromatin and proteomic biomarkers when comparing patients that for which we observed CTCAE grades of 3 to those with smaller grades. Since all grade 3 patients are patients with Head & Neck tumors, we will restrict our analyses to those.

In [ ]:
markers = [
    "volume",
    "hetero_to_euchromatin_area_ratio",
    "radial_chromatin_content_p10",
    "std_curvature",
    "glcm_contrast_5px",
    "glcm_correlation_5px",
]
marker_labels = [
    r"Nuclear volume in px$^3$",
    "relative HC/EC ratio",
    "Fraction of the overall DNA intensity \n within the inner 10% of the nuclear volume",
    "Standard deviation of the curvature",
    "Contrast of the GLCM \n with a shift of 5px (2D)",
    "Correlation of the GLCM \n with a shift of 5px (2D)",
]
hn_data = data.loc[data.cancer == "Head & Neck"]
for i in range(len(markers)):
    fig, ax = plt.subplots(figsize=[6, 4])
    ax = sns.violinplot(data=hn_data, x="toxic", y=markers[i], order=["no", "yes"])
    ax.set_ylabel(marker_labels[i])
    ax.set_xlabel("CTCAE grade >= 3")
    annotator = Annotator(
        ax,
        [("no", "yes")],
        data=data,
        x="toxic",
        y=markers[i],
        order=["no", "yes"],
        plot="violinplot",
    )
    annotator.configure(
        test="t-test_welch",
        text_format="simple",
        loc="inside",
        comparisons_correction="Benjamini-Hochberg",
    )
    annotator.apply_test()
    annotator.annotate()
    plt.show()
    plt.close()

In [ ]:
markers = [
    "rel_lamin_3d_int",
    "rel_gh2ax_3d_int",
    "gh2ax_foci_count",
    "gh2ax_sum_foci_area",
    "gh2ax_avg_foci_area",
]
marker_labels = [
    "Volume-normalized nuclear\nLamin A/C intensity",
    "Normalized nuclear\n" r"$\gamma$H2AX intensity",
    r"Number of $\gamma$H2AX foci",
    r"Sum of the $\gamma$H2AX foci area",
    r"Average size of the $\gamma$H2AX foci",
]
hn_data = data.loc[data.cancer == "Head & Neck"]
for i in range(len(markers)):
    fig, ax = plt.subplots(figsize=[6, 4])
    ax = sns.barplot(data=hn_data, x="toxic", y=markers[i], order=["no", "yes"])
    ax.set_ylabel(marker_labels[i])
    ax.set_xlabel("CTCAE grade >= 3")
    annotator = Annotator(
        ax,
        [("no", "yes")],
        data=data,
        x="toxic",
        y=markers[i],
        order=["no", "yes"],
        plot="barplot",
    )
    annotator.configure(
        test="t-test_welch",
        text_format="simple",
        loc="inside",
        comparisons_correction="Benjamini-Hochberg",
    )
    annotator.apply_test()
    annotator.annotate()

In [ ]:
groupkfold = StratifiedGroupKFold(n_splits=5)
rfc = RandomForestClassifier(
    n_estimators=500, n_jobs=10, random_state=seed, class_weight="balanced"
)

hn_data = chrometric_data.loc[
    np.logical_and(cancer_labels == "Head & Neck", tp_labels != "prior")
]
hn_toxic_labels = data.loc[hn_data.index, "toxic"]
hn_sample_labels = data.loc[hn_data.index, "sample"]

toxic_hn_cv_bacs = cross_val_score(
    rfc,
    cv=groupkfold,
    X=np.array(hn_data),
    y=np.array(hn_toxic_labels),
    groups=np.array(hn_sample_labels),
    scoring="balanced_accuracy",
    n_jobs=10,
)
print(
    "Balanced accuracy: {} (+/- {})".format(
        np.mean(toxic_hn_cv_bacs), np.std(toxic_hn_cv_bacs)
    )
)

In [ ]:
test_samples = []
for train_idx, test_idx in groupkfold.split(hn_data, hn_toxic_labels, hn_sample_labels):
    test_samples.append("_".join(np.unique(hn_sample_labels[test_idx])))
cv_bac_df = pd.DataFrame(toxic_hn_cv_bacs, columns=["BAC"])
cv_bac_df["Test samples"] = test_samples
fig, ax = plt.subplots(figsize=[4, 4])
ax = sns.barplot(x="Test samples", y="BAC", data=cv_bac_df, palette=["gray"], ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.title("Test set performance by fold")
plt.yticks(np.arange(0, 1, 0.1))
plt.show()

In [ ]:
toxic_patients

---

#### 4.1.b. Assessment of the chrometric differences for patients with experiencing fatigue

We next will assess whether or not the chrometric data enables the distinction between patients that experience fatigue an those that do not. To assess that we will train a RandomForest classifier on trying to predict whether or not the CTCAE grade for fatigue of a patient is greater than 0 from the chrometric profiles of a given PBMC.

Note that we only observe patients suffering from fatigue among the Meningioma and Glioma but not among the Head & Neck cancer patients. We thus subset the data to those. Since, we observe in total 9 of the 20 Glioma and Meningioma patients to suffer from fatigue and thus the data is approximately balanced we do not further subsample the data.

In [ ]:
mggl_data = chrometric_data.loc[
    np.logical_and(cancer_labels != "Head & Neck", tp_labels != "prior")
]
mggl_fatigue_labels = data.loc[mggl_data.index, "fatigue"] > 0
mggl_sample_labels = data.loc[mggl_data.index, "sample"]

In [ ]:
fatigue_cv_conf_mtx_patient = get_cv_conf_mtx(
    estimator=rfc,
    features=mggl_data,
    labels=np.array(mggl_fatigue_labels),
    groups=np.array(mggl_sample_labels),
    scale_features=False,
    n_folds=5,
    # order=[False, True],
)
normalized_cv_conf_mtx_patient = fatigue_cv_conf_mtx_patient.divide(
    fatigue_cv_conf_mtx_patient.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_patient,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
)
ax.set_xlabel("Predicted fatigue")
ax.set_ylabel("True fatigue")
plt.show()

In [ ]:
groupkfold = StratifiedGroupKFold(n_splits=5)
fatigue_cv_bacs = cross_val_score(
    rfc,
    cv=groupkfold,
    X=np.array(mggl_data),
    y=np.array(mggl_fatigue_labels),
    groups=np.array(mggl_sample_labels),
    scoring="balanced_accuracy",
    n_jobs=10,
)
print(
    "Balanced accuracy: {} (+/- {})".format(
        np.mean(fatigue_cv_bacs), np.std(fatigue_cv_bacs)
    )
)

In [ ]:
test_samples = []
for train_idx, test_idx in groupkfold.split(
    mggl_data, mggl_fatigue_labels, mggl_sample_labels
):
    test_samples.append("_".join(np.unique(mggl_sample_labels[test_idx])))
cv_bac_df = pd.DataFrame(fatigue_cv_bacs, columns=["BAC"])
cv_bac_df["Test samples"] = test_samples
fig, ax = plt.subplots(figsize=[4, 4])
ax = sns.barplot(x="Test samples", y="BAC", data=cv_bac_df, palette=["gray"], ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.title("Test set performance by fold")
plt.yticks(np.arange(0, 1, 0.1))
plt.show()

In [ ]:
set(mggl_sample_labels[mggl_fatigue_labels == True])

---

#### 4.1.b. Assessment of the chrometric differences for patients with experiencing dermatitis radiation

We next will assess whether or not the chrometric data enables the distinction between patients that experience dermatitis an those that do not. To assess that we will train a RandomForest classifier on trying to predict whether or not the CTCAE grade for dermatitis of a patient is greater than 0 from the chrometric profiles of a given PBMC.

To this end, we again exclude all Head & Neck cancer patient samples as 9/10 of those have experienced dermatitis.

In [ ]:
mggl_derm_labels = data.loc[mggl_data.index, "dermatitis radiation"] > 0

In [ ]:
derm_cv_conf_mtx_patient = get_cv_conf_mtx(
    estimator=rfc,
    features=mggl_data,
    labels=np.array(mggl_derm_labels),
    groups=np.array(mggl_sample_labels),
    scale_features=False,
    n_folds=5,
    # order=[False, True],
)
normalized_cv_conf_mtx_patient = derm_cv_conf_mtx_patient.divide(
    derm_cv_conf_mtx_patient.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_patient,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
)
ax.set_xlabel("Predicted dermatitis")
ax.set_ylabel("True dermatitis")
plt.show()

In [ ]:
derm_cv_bacs = cross_val_score(
    rfc,
    cv=groupkfold,
    X=np.array(mggl_data),
    y=np.array(mggl_derm_labels),
    groups=np.array(mggl_sample_labels),
    scoring="balanced_accuracy",
    n_jobs=10,
)
print(
    "Balanced accuracy: {} (+/- {})".format(np.mean(derm_cv_bacs), np.std(derm_cv_bacs))
)

In [ ]:
test_samples = []
for train_idx, test_idx in groupkfold.split(
    mggl_data, mggl_derm_labels, mggl_sample_labels
):
    test_samples.append("_".join(np.unique(mggl_sample_labels[test_idx])))
cv_bac_df = pd.DataFrame(derm_cv_bacs, columns=["BAC"])
cv_bac_df["Test samples"] = test_samples
fig, ax = plt.subplots(figsize=[4, 4])
ax = sns.barplot(x="Test samples", y="BAC", data=cv_bac_df, palette=["gray"], ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.title("Test set performance by fold")
plt.yticks(np.arange(0, 1, 0.1))
plt.show()

In [ ]:
set(mggl_sample_labels[mggl_derm_labels == True])